# link to cloud

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Package

In [16]:
import numpy as np
import torch
import torchvision
import matplotlib
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from sklearn import preprocessing
from enum import Enum 
import copy
# import SGD for optimizer
from torch.optim import SGD

# import Adam for optimizer
from torch.optim import Adam

# to measure the performance import L1Loss
from torch.nn import L1Loss

# import data

In [17]:
training_y = np.load('/content/drive/MyDrive/code/python/projects/ADL/ADL_(IM5062)/111ntu-homework2/training_y.npy')

In [18]:
training_y

array([[45., 40.],
       [45., 35.],
       [40., 35.],
       ...,
       [30., 25.],
       [46., 38.],
       [62., 42.]])

In [4]:
# training_y[87]

In [5]:
# print(np.size(training_y))
# print(len(training_y))

In [6]:
# training_y[0]

In [7]:
training_x_grid = np.load('/content/drive/MyDrive/code/python/projects/ADL/ADL_(IM5062)/111ntu-homework2/training_x_grid.npy')

In [19]:
# training_x_grid
#print(np.size(training_x_grid))
#print(len(training_x_grid))
#training_x_grid[-1]
#training_x_grid[0][0][0]
#training_x_grid[0][0][0]
#len(training_x_grid[0][0])
#len(training_x_grid[0])
#training_x_grid[0][0][0]

# custom dataset 

In [20]:
class CustomDataset(Dataset):
    # This loads the data and converts it, make data rdy
    def __init__(self):
        # load data
        self.df= training_x_grid
        # extract labels
        self.df_labels= training_y
        # conver to torch dtypes
        self.dataset=torch.tensor(self.df).float()

        self.labels=torch.tensor(self.df_labels).long()
    
    # This returns the total amount of samples in your Dataset
    def __len__(self):
        return len(self.dataset)
    
    # This returns given an index the i-th sample and label
    def __getitem__(self, idx):
        return self.dataset[idx],self.labels[idx]

In [21]:
ds=CustomDataset()
ds

# Training and Validation Datasets


In [22]:
from torch.utils.data import random_split
# 8760, 7665(115*73), 1095(5*73) 
train_ds, val_ds = random_split(ds, [8395, 365])
len(train_ds), len(val_ds)

(8395, 365)

In [23]:
#train_ds[0]
#val_ds[0]
#val_ds[0]
#val_ds[0]

In [24]:
from torch.utils.data import DataLoader

batch_size = 73

train_loader = DataLoader(train_ds,batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size, num_workers=2, pin_memory=True)

In [25]:
# for images, labels in val_loader:
#     print('images.shape:', images.shape)
    # out = simple_model(images)
    # print('out.shape:', out.shape)

In [26]:
# for images, labels in train_loader:
#     print('images.shape:', images.shape)
#     # out = simple_model(images)
#     # print('out.shape:', out.shape)
#     break

# testing datasets

In [27]:
testing_x_grid = np.load('/content/drive/MyDrive/code/python/projects/ADL/ADL_(IM5062)/111ntu-homework2/testing_x_grid.npy')

In [28]:
#testing_x_grid 

In [29]:
testing_x_grid_T = torch.tensor(testing_x_grid.astype(np.float32))

In [30]:
# testing_x_grid_T

In [31]:
# testing_x_grid_T.shape

#test with simple model

In [32]:
simple_model = nn.Sequential(
            nn.Conv2d(30, 32, kernel_size=2, padding=1),
            #input 30 channel(RGB) and output 32
            # output: 32  38, 13
            nn.ReLU(),  # doesn't change shape
            # output: 32  38, 13
            nn.Conv2d(32, 64, kernel_size=2, stride=1, padding=1),
            # output: 64  38, 13
            nn.ReLU(),
            # output: 64  38, 13
            nn.MaxPool2d(2, 2), # output: 64 x 19 x 7

            nn.Conv2d(64, 128, kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 128 x 11, 4

            nn.Conv2d(256, 512, kernel_size=2, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 512, 6, 2

            # nn.Flatten(),
            # nn.Linear(8512, 64),
            # nn.ReLU(),
            # nn.Linear(64, 2)
)

In [33]:
for images, labels in train_loader:
    print('images.shape:', images.shape)
    out = simple_model(images)
    print('out.shape:', out.shape)
    print(out.shape)

    # print("-------")
    # print(labels.shape)

    # # loss_fc = L1Loss()
    # loss_fc = nn.MSELoss()
    # loss = loss_fc(out, labels)
    # print(loss)
    break

  

images.shape: torch.Size([73, 30, 38, 13])
out.shape: torch.Size([73, 512, 6, 2])
torch.Size([73, 512, 6, 2])


# model


In [34]:
from torch.nn import L1Loss
class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss_fc1 = L1Loss()
        #loss_fc1 = nn.MSELoss()
        loss = loss_fc1(out, labels)
        # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss_fc2 = L1Loss()
        #loss_fc2 = nn.MSELoss()
        loss2 = loss_fc2(out, labels)   
        # Calculate loss
        return {'val_loss': loss2.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss']))
      

In [35]:
# loss = nn.MSELoss()

In [36]:
class Cifar10CnnModel(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            # input 30, 38, 13
            nn.Conv2d(30, 32, kernel_size=2, padding=1),
            #input 30 channel(RGB) and output 32
            # output: 32  38, 13
            nn.ReLU(),  # doesn't change shape
            # output: 32  38, 13
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            # output: 64  38, 13
            nn.ReLU(),
            # output: 64  38, 13
            nn.MaxPool2d(2, 2), # output: 64 x 19 x 7

            nn.Flatten(), 

            nn.Linear(8512,512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(512,256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(256, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.4),

            nn.Linear(64, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),

            nn.Linear(64, 2))
        
    def forward(self, xb):
        return self.network(xb)

In [37]:
model = Cifar10CnnModel()
model

Cifar10CnnModel(
  (network): Sequential(
    (0): Conv2d(30, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Flatten(start_dim=1, end_dim=-1)
    (6): Linear(in_features=8512, out_features=512, bias=True)
    (7): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Dropout(p=0.4, inplace=False)
    (10): Linear(in_features=512, out_features=256, bias=True)
    (11): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): Dropout(p=0.4, inplace=False)
    (14): Linear(in_features=256, out_features=64, bias=True)
    (15): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (16): ReLU()
    (17): Dropout(p=0.4, inplace=False)
    (18): Linear(in_fe

In [38]:
@torch.no_grad()  # means that will not do any gradients while evaluate
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    #print("1")
    for epoch in range(epochs):
        # Training Phase 
        #print("2")
        model.train()
        #print("3")
        train_losses = []
        for batch in train_loader:
            #print("4")
            loss = model.training_step(batch)
            #print("5")
            train_losses.append(loss)
            #print("6")
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [39]:
model = Cifar10CnnModel()
model

Cifar10CnnModel(
  (network): Sequential(
    (0): Conv2d(30, 32, kernel_size=(2, 2), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Flatten(start_dim=1, end_dim=-1)
    (6): Linear(in_features=8512, out_features=512, bias=True)
    (7): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Dropout(p=0.4, inplace=False)
    (10): Linear(in_features=512, out_features=256, bias=True)
    (11): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU()
    (13): Dropout(p=0.4, inplace=False)
    (14): Linear(in_features=256, out_features=64, bias=True)
    (15): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (16): ReLU()
    (17): Dropout(p=0.4, inplace=False)
    (18): Linear(in_fe

In [42]:
evaluate(model, val_loader)

{'val_loss': 23.63582420349121}

In [43]:
num_epochs = 20
opt_func = torch.optim.Adam
lr = 0.001

In [44]:
history = fit(num_epochs, lr, model, train_loader, val_loader, opt_func)

Epoch [0], train_loss: 20.3536, val_loss: 19.7669
Epoch [1], train_loss: 14.0597, val_loss: 12.6637
Epoch [2], train_loss: 7.5851, val_loss: 5.9990
Epoch [3], train_loss: 5.6569, val_loss: 5.3842
Epoch [4], train_loss: 5.3414, val_loss: 7.6355
Epoch [5], train_loss: 5.2622, val_loss: 4.8771
Epoch [6], train_loss: 5.2129, val_loss: 5.3913
Epoch [7], train_loss: 5.1669, val_loss: 5.1047
Epoch [8], train_loss: 5.1440, val_loss: 5.0809
Epoch [9], train_loss: 4.9964, val_loss: 5.6654
Epoch [10], train_loss: 5.0639, val_loss: 4.8119
Epoch [11], train_loss: 4.9756, val_loss: 4.7567
Epoch [12], train_loss: 5.0092, val_loss: 4.7728
Epoch [13], train_loss: 5.0081, val_loss: 4.7238
Epoch [14], train_loss: 4.9998, val_loss: 4.9243
Epoch [15], train_loss: 4.9498, val_loss: 4.7377
Epoch [16], train_loss: 4.8985, val_loss: 4.9931
Epoch [17], train_loss: 4.9957, val_loss: 4.6043
Epoch [18], train_loss: 4.8882, val_loss: 4.8976
Epoch [19], train_loss: 4.9047, val_loss: 4.7126


In [63]:
num_epochs = 20
opt_func = torch.optim.Adam
lr = 0.0005

In [64]:
history = fit(num_epochs, lr, model, train_loader, val_loader, opt_func)

Epoch [0], train_loss: 4.8059, val_loss: 4.6497
Epoch [1], train_loss: 4.7532, val_loss: 4.5610
Epoch [2], train_loss: 4.7205, val_loss: 5.0517
Epoch [3], train_loss: 4.7280, val_loss: 4.5423
Epoch [4], train_loss: 4.7170, val_loss: 4.5384
Epoch [5], train_loss: 4.7473, val_loss: 4.8012
Epoch [6], train_loss: 4.7263, val_loss: 4.5878
Epoch [7], train_loss: 4.7281, val_loss: 4.5676
Epoch [8], train_loss: 4.6323, val_loss: 5.1381
Epoch [9], train_loss: 4.6957, val_loss: 4.6792
Epoch [10], train_loss: 4.5879, val_loss: 4.5504
Epoch [11], train_loss: 4.5970, val_loss: 4.9292
Epoch [12], train_loss: 4.6788, val_loss: 4.6134
Epoch [13], train_loss: 4.6022, val_loss: 4.6070
Epoch [14], train_loss: 4.6375, val_loss: 4.7654
Epoch [15], train_loss: 4.5861, val_loss: 4.5672
Epoch [16], train_loss: 4.6113, val_loss: 4.8600
Epoch [17], train_loss: 4.5871, val_loss: 4.5598
Epoch [18], train_loss: 4.5142, val_loss: 4.9464
Epoch [19], train_loss: 4.4995, val_loss: 5.3190


In [65]:
num_epochs = 20
opt_func = torch.optim.Adam
lr = 0.0001

In [66]:
history += fit(num_epochs, lr, model, train_loader, val_loader, opt_func)

Epoch [0], train_loss: 4.4182, val_loss: 4.4606
Epoch [1], train_loss: 4.4462, val_loss: 4.4405
Epoch [2], train_loss: 4.4317, val_loss: 4.6320
Epoch [3], train_loss: 4.4289, val_loss: 4.5212
Epoch [4], train_loss: 4.4129, val_loss: 4.4778
Epoch [5], train_loss: 4.3381, val_loss: 4.4413
Epoch [6], train_loss: 4.4392, val_loss: 4.4607
Epoch [7], train_loss: 4.3733, val_loss: 4.4263
Epoch [8], train_loss: 4.4371, val_loss: 4.4382
Epoch [9], train_loss: 4.3579, val_loss: 4.4581
Epoch [10], train_loss: 4.4230, val_loss: 4.4134
Epoch [11], train_loss: 4.3543, val_loss: 4.4151
Epoch [12], train_loss: 4.3730, val_loss: 4.5137
Epoch [13], train_loss: 4.3406, val_loss: 4.4698
Epoch [14], train_loss: 4.3850, val_loss: 4.4376
Epoch [15], train_loss: 4.3761, val_loss: 4.4054
Epoch [16], train_loss: 4.3057, val_loss: 4.4729
Epoch [17], train_loss: 4.2999, val_loss: 4.4994
Epoch [18], train_loss: 4.3662, val_loss: 4.4376
Epoch [19], train_loss: 4.3278, val_loss: 4.4115


In [67]:
num_epochs = 10
opt_func = torch.optim.Adam
lr = 0.00005

In [68]:
history += fit(num_epochs, lr, model, train_loader, val_loader, opt_func)

Epoch [0], train_loss: 4.4133, val_loss: 4.3129
Epoch [1], train_loss: 4.4119, val_loss: 4.3103
Epoch [2], train_loss: 4.4197, val_loss: 4.3194
Epoch [3], train_loss: 4.4170, val_loss: 4.3425
Epoch [4], train_loss: 4.3945, val_loss: 4.3214
Epoch [5], train_loss: 4.3979, val_loss: 4.3404
Epoch [6], train_loss: 4.3623, val_loss: 4.2920
Epoch [7], train_loss: 4.3801, val_loss: 4.3118
Epoch [8], train_loss: 4.3445, val_loss: 4.3068
Epoch [9], train_loss: 4.3447, val_loss: 4.3015


In [69]:
num_epochs = 10
opt_func = torch.optim.Adam
lr = 0.000001

In [70]:
history += fit(num_epochs, lr, model, train_loader, val_loader, opt_func)

Epoch [0], train_loss: 4.3622, val_loss: 4.3089
Epoch [1], train_loss: 4.3837, val_loss: 4.3001
Epoch [2], train_loss: 4.4061, val_loss: 4.3128
Epoch [3], train_loss: 4.3487, val_loss: 4.3042
Epoch [4], train_loss: 4.3589, val_loss: 4.3119
Epoch [5], train_loss: 4.3793, val_loss: 4.3097
Epoch [6], train_loss: 4.3683, val_loss: 4.3025
Epoch [7], train_loss: 4.3987, val_loss: 4.3093
Epoch [8], train_loss: 4.3607, val_loss: 4.3184
Epoch [9], train_loss: 4.3726, val_loss: 4.2989


In [45]:
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

In [40]:
# plot_losses(history)

# save model

In [68]:
FILE = 'hw2_8_8.pt'


In [41]:
best = '/content/drive/MyDrive/code/python/projects/ADL/ADL_(IM5062)/111ntu-homework2/R10723059_胡祖望.pt'

In [69]:
torch.save(model, FILE)

In [70]:
model = torch.load(FILE)

In [42]:
model = torch.load(best)

# output to csv


In [43]:
out = model(testing_x_grid_T)

In [44]:
out 

tensor([[77.4743, 63.1286],
        [81.7124, 66.4834],
        [78.5043, 63.8757],
        ...,
        [61.5300, 50.1782],
        [57.6184, 46.9490],
        [50.3672, 42.0186]], grad_fn=<AddmmBackward0>)

In [45]:
out.shape

torch.Size([8760, 2])

In [46]:
import pandas as pd
import os.path
ans_np = out.detach().numpy() #convert to Numpy array
df = pd.DataFrame(ans_np) #convert to a dataframe


In [47]:
df

,0,1
0,77.474297,63.128574
1,81.712448,66.483421
2,78.504318,63.875706
3,62.528881,51.736996
4,53.446686,44.884949
...,...,...
8755,70.968781,56.857479
8756,68.695305,55.251698
8757,61.530003,50.178188
8758,57.618420,46.948994


In [48]:
df.rename(columns = {0:'PM2.5-1'}, inplace = True)
df.rename(columns = {1:'PM2.5-2'}, inplace = True)
df.to_csv(os.path.join('/content/drive/MyDrive/code/python/projects/ADL/ADL_(IM5062)/111ntu-homework2','ans8_8.csv')) #save to file
#df.rename(columns = {:'INDEX'}, inplace = True)


In [49]:
df3 = pd.read_csv('/content/drive/MyDrive/code/python/projects/ADL/ADL_(IM5062)/111ntu-homework2/ans8_8.csv')

In [50]:
df3

,Unnamed: 0,PM2.5-1,PM2.5-2
0,0,77.474300,63.128574
1,1,81.712450,66.483420
2,2,78.504320,63.875706
3,3,62.528880,51.736996
4,4,53.446686,44.884950
...,...,...,...
8755,8755,70.968780,56.857480
8756,8756,68.695305,55.251698
8757,8757,61.530003,50.178190
8758,8758,57.618420,46.948994


In [51]:
df3.rename(columns = {'Unnamed: 0':'INDEX'}, inplace = True)
df3['INDEX'] = df3['INDEX']+1

In [52]:
df3

,INDEX,PM2.5-1,PM2.5-2
0,1,77.474300,63.128574
1,2,81.712450,66.483420
2,3,78.504320,63.875706
3,4,62.528880,51.736996
4,5,53.446686,44.884950
...,...,...,...
8755,8756,70.968780,56.857480
8756,8757,68.695305,55.251698
8757,8758,61.530003,50.178190
8758,8759,57.618420,46.948994


In [53]:
df3.to_csv(os.path.join('/content/drive/MyDrive/code/python/projects/ADL/ADL_(IM5062)/111ntu-homework2','ans8_8.csv'),index=False) #save to file

In [54]:
df4 = pd.read_csv('/content/drive/MyDrive/code/python/projects/ADL/ADL_(IM5062)/111ntu-homework2/ans8_8.csv')

In [55]:
df4

,INDEX,PM2.5-1,PM2.5-2
0,1,77.474300,63.128574
1,2,81.712450,66.483420
2,3,78.504320,63.875706
3,4,62.528880,51.736996
4,5,53.446686,44.884950
...,...,...,...
8755,8756,70.968780,56.857480
8756,8757,68.695305,55.251698
8757,8758,61.530003,50.178190
8758,8759,57.618420,46.948994
